## Inisiasi 

In [9]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
from newspaper import Article
import nltk
import sys
from IPython.display import clear_output

host = '127.0.0.1'
port = "3306"
user = 'root'
password = 'ugm2018'
database = 'gdelt2'

table_name = 'events_SN'
fips_country_code = 'SN'
country_code = 'SGP'
# IDN, MYS, THA, SGP, PHL

conn_str = 'mysql+pymysql://{}:{}@{}:{}/{}'.format(user, password, host, port, database);           
engine = create_engine(conn_str)

def run(sql):
    df = pd.read_sql_query(sql,engine)
    return df

def execute(sql):
#     print(sql)
    return engine.execute(statement=sql)


## Add Field to Store Keyword in DB ( lakukan sekali saja)

In [11]:
# execute("""
# ALTER TABLE {} 
# ADD COLUMN `keywords` TEXT NULL AFTER `SOURCEURL`;
# """.format(table_name))

execute("""
ALTER TABLE {} 
ADD COLUMN `title` TEXT NULL AFTER `keywords`;
""".format(table_name))

## Load GDELT Data

In [12]:
# load data tahun 2014 ke atas dengan nilai AvgTone > 8 (berita bagus)
# select GLOBALEVENTID, SOURCEURL,NumArticles, NumSources, AvgTone
# from {}
# where
#     year > 2014 and AvgTone< 8 order by NumSources DESC, NumArticles DESC LIMIT 
    
news = run("""
SELECT * FROM
(SELECT t1.GLOBALEVENTID, t1.SOURCEURL, t1.NumArticles,t1.NumSources, t1.AvgTone,
  ROW_NUMBER() OVER(PARTITION BY t1.MonthYear ORDER BY t1.MonthYear, t1.NumSources DESC, t1.NumArticles DESC) AS Counter
  FROM {} t1
  WHERE t1.year > 2014 and (t1.AvgTone<-8 OR t1.AvgTone>8)) t2 WHERE t2.Counter < 35
""".format(table_name))

news.to_csv('scrapping_SN.csv')


In [6]:
def escape(str):
    return ''.join(e for e in str if e.isalnum())

In [13]:
news.shape

(1224, 6)

## It's time to scrap

In [14]:
errorCount = 0
successCount = 0
counter = 0
import re

t = len(news.values);

for item in news.values:
    counter += 1
    if (counter % 10 == 0):
        clear_output(wait=True)
        
    try:        
        url = item[1]              
        print("Scrapping ", url, " -> ", str(counter)," of ", str(t)," -> ", round(counter/t * 100, 2), " %")        
        article = Article(url)
        article.download()

        article.parse()
        article.nlp()

        execute("""
        update {} SET keywords="{}", title="{}"         
        where
            GLOBALEVENTID = {}
        """.format(table_name, ','.join(article.keywords), escape(article.title), item[0]))

        with open('news/'+country_code+'/'+str(item[0])+'.txt', 'w') as outfile:              
            outfile.write(article.text)
#             clear_output(wait=True)
            print('Scraping ',url,'Done..')            
#             clear_output(wait=True)
        successCount += 1
    except:
        errorCount += 1        
        print("Error scrapping:", sys.exc_info())        
                  
#     clear_output(wait=True)              
    print('success:',successCount)
    print('error:',errorCount)
    

Scrapping  http://www.themalaymailonline.com/world/article/singapores-shanmugam-backs-polices-raid-tactics-on-illegal-brothels  ->  1220  of  1224  ->  99.67  %
Scraping  http://www.themalaymailonline.com/world/article/singapores-shanmugam-backs-polices-raid-tactics-on-illegal-brothels Done..
success: 927
error: 293
Scrapping  https://www.msn.com/en-sg/news/world/fugitive-vietnam-agent-concerned-for-his-safety-if-sent-home/ar-BBHPxPS  ->  1221  of  1224  ->  99.75  %
Error scrapping: (<class 'newspaper.article.ArticleException'>, ArticleException('Article `download()` failed with 410 Client Error: Gone for url: https://www.msn.com/en-sg/news/world/fugitive-vietnam-agent-concerned-for-his-safety-if-sent-home/ar-BBHPxPS on URL https://www.msn.com/en-sg/news/world/fugitive-vietnam-agent-concerned-for-his-safety-if-sent-home/ar-BBHPxPS'), <traceback object at 0x7eff864c57c8>)
success: 927
error: 294
Scrapping  http://borneobulletin.com.bn/fugitive-vietnam-agent-concerned-for-his-safety-if-